In [2]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [3]:
cudnn.benchmark = True
cudnn.deterministic = False

In [4]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [5]:
opt = get_config("config_files/train.yaml")

In [6]:
import torch

# Create a device object to represent the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
if torch.cuda.is_available():
    
    print("CUDA is available")
else:
    print("CUDA is not available")

In [7]:
train(opt, amp=False).to(device)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /home/musa/Desktop/Work/Mina/Project3/EasyOCR-master/trainer/all_data
opt.select_data: ['/home/musa/Desktop/Work/Mina/Project3/EasyOCR', 'master/trainer/all_data/train']
opt.batch_ratio: ['10162', '2591']
--------------------------------------------------------------------------------
dataset_root:    /home/musa/Desktop/Work/Mina/Project3/EasyOCR-master/trainer/all_data	 dataset: /home/musa/Desktop/Work/Mina/Project3/EasyOCR
/home/musa/Desktop/Work/Mina/Project3/EasyOCR-master/trainer/all_data/val
       filename words
0     10445.jpg  1813
1     10714.jpg  1801
2     10819.jpg    22
3     11635.jpg  1806
4     11720.jpg  1808
...         ...   ...
2627  11010.jpg  1819
2628  11949.jpg  1827
2629  11498.jpg  1823
2630  11557.jpg  1801
2631  10504.jpg  

/home/musa/anaconda3/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/musa/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/musa/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/musa/anaconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/musa/anaconda3/lib/python3.10/site-packages/torch/utils/data/dataset.py", line 298, in __getitem__
    return self.dataset[self.indices[idx]]
  File "/home/musa/anaconda3/lib/python3.10/site-packages/torch/utils/data/dataset.py", line 243, in __getitem__
    return self.datasets[dataset_idx][sample_idx]
  File "/home/musa/Desktop/Work/Mina/Project3/EasyOC

In [ ]:
"""# Move the model to the GPU
model = train(opt, amp=False).to(device)

# Move the data to the GPU
# Here you should replace this with your own data loading and preprocessing code
data = torch.randn(32, 3, 224, 224).to(device)

# Train the model on the GPU
model.train()
for i in range(100):
    output = model(data)
    loss = output.mean()
    loss.backward()"""

In [ ]:
print("finish")